In [1]:
import sys
import os
import boto3
import rasterio


from rasterio.merge import merge

# Import raster helpers
sys.path.insert(0, "/home/wb411133/Code/gostrocks/src")

import GOSTRocks.rasterMisc as rMisc
from GOSTRocks.misc import tPrint

# Import GOST urban functions
sys.path.append("../../../src")

# Import local functions
from novelUrbanization import *

%load_ext autoreload
%autoreload 2

s3client = boto3.client("s3")

/home/wb411133/.conda/envs/ee/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


METADATA Library: Could not import arcgis libraries


# Mosaic the DoU layers

In [2]:
bucket = "wbg-geography01"
prefix = "URBANIZATION/MR_Novel_Poverty/"

# Loop through the S3 bucket and get all the keys for files that are .tif
more_results = True
loops = 0
urban_tiff = {}

while more_results:
    if loops > 0:
        objects = s3client.list_objects_v2(
            Bucket=bucket, Prefix=prefix, ContinuationToken=token
        )
    else:
        objects = s3client.list_objects_v2(Bucket=bucket, Prefix=prefix)
    more_results = objects["IsTruncated"]
    if more_results:
        token = objects["NextContinuationToken"]
    loops += 1
    for res in objects["Contents"]:
        if ("FINAL_STANDARD_1KM" in res["Key"]) and (
            res["Key"].endswith("urban.tif") or res["Key"].endswith("urban_hd.tif")
        ):
            cur_pop = os.path.basename(res["Key"]).split("_")[1]
            cur_type = os.path.basename(res["Key"]).split("_")[-1].replace(".tif", "")
            cur_lyr = f"{cur_pop}_{cur_type}"
            cur_path = os.path.join("s3://", bucket, res["Key"])
            try:
                urban_tiff[cur_lyr].append(cur_path)
            except:
                urban_tiff[cur_lyr] = [cur_path]

In [3]:
out_folder = "/home/wb411133/temp/urban_mosaics"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

for cur_lbl, rasters in urban_tiff.items():
    tPrint(cur_lbl)
    out_file = os.path.join(out_folder, f"{cur_lbl}_DoU_Africa_mosaic.tif")
    curD, profile = rMisc.merge_rasters(
        rasters, merge_method="max", boolean_gt_0=True, out_file=out_file
    )

15:06:26	cpo15_urban
15:06:44	cpo15_hd
15:07:06	cpo20_urban
15:07:30	cpo20_hd
15:07:53	gpo_urban
15:08:22	gpo_hd
15:08:50	upo15_urban
15:09:42	upo15_hd
15:10:31	upo18_urban
15:10:34	upo18_hd
15:10:36	upo16_urban
15:10:38	upo16_hd
15:10:39	cpo17_urban
15:10:40	cpo17_hd
15:10:40	upo17_urban
15:10:41	upo17_hd
15:10:41	cpo18_urban
15:10:42	cpo18_hd


In [ ]:
opened_tiffs = [rasterio.open(x) for x in hd_urban_tiff]
merged, out_transform = merge(opened_tiffs, method="max")
merged = ((merged > 0) * 1).astype("uint8")

# Create a new raster file with the merged data
metadata = opened_tiffs[0].meta.copy()
metadata.update(
    {
        "height": merged.shape[1],
        "width": merged.shape[2],
        "transform": out_transform,
        "dtype": "uint8",
    }
)
with rasterio.open(
    f"/home/wb411133/temp/{sel_pop}_combo_urban_hd.tif", "w", **metadata
) as dst:
    dst.write(merged)

# Mosaick data from PP

In [28]:
bucket = "wbg-geography01"
prefix = "URBANIZATION/MR_Novel_Poverty/AAPPC/Delineations/"

# Loop through the S3 bucket and get all the keys for files that are .tif
more_results = True
loops = 0
db_tiffs = {}

while more_results:
    if loops > 0:
        objects = s3client.list_objects_v2(
            Bucket=bucket, Prefix=prefix, ContinuationToken=token
        )
    else:
        objects = s3client.list_objects_v2(Bucket=bucket, Prefix=prefix)
    more_results = objects["IsTruncated"]
    if more_results:
        token = objects["NextContinuationToken"]
    loops += 1
    for res in objects["Contents"]:
        if "1k" in res["Key"]:
            cur_path = os.path.join("s3://", bucket, res["Key"])
            cur_type = os.path.basename(cur_path).split("_")[2][:2]
            if cur_type in ["cc", "co", "ur"]:
                cur_pop = os.path.basename(cur_path).split("_")[1].split("d")[0]
                cur_label = f"{cur_pop}_{cur_type}"
                try:
                    db_tiffs[cur_label].append(cur_path)
                except:
                    db_tiffs[cur_label] = [cur_path]

In [30]:
out_folder = "/home/wb411133/temp/urban_mosaics"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

for cur_lbl, rasters in urban_tiff.items():
    tPrint(cur_lbl)
    out_file = os.path.join(out_folder, f"{cur_lbl}_DB_Africa_mosaic.tif")
    curD, profile = rMisc.merge_rasters(
        rasters, merge_method="max", boolean_gt_0=True, out_file=out_file
    )

15:18:39	cpo15_urban
15:19:04	cpo15_hd
15:19:27	cpo20_urban
15:19:53	cpo20_hd
15:20:16	gpo_urban
15:20:44	gpo_hd
15:21:11	upo15_urban
15:21:56	upo15_hd
15:22:39	upo18_urban
15:22:41	upo18_hd
15:22:44	upo16_urban
15:22:45	upo16_hd
15:22:46	cpo17_urban
15:22:47	cpo17_hd
15:22:47	upo17_urban
15:22:47	upo17_hd
15:22:48	cpo18_urban
15:22:48	cpo18_hd
